In [ ]:
import os, sys
from tqdm import trange

import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy

import torch
from torch import nn
from torch.utils.data import TensorDataset

source = "/home/loek/projects/rnn/source"
sys.path.append(source)

from data import fun_data, grid_data
from preprocessing import Direct, Encoding, OneHot
from compilation import Compiler, Tracker, ScalarTracker, ActivationTracker
from activations import get_activations
from data_analysis.automata import to_automaton_history
from data_analysis.visualization.animation import SliderAnimation
from data_analysis.visualization.activations import (
    ActivationsAnimation,
    FunctionAnimation,
)
from data_analysis.visualization.automata import AutomatonAnimation
from data_analysis.visualization.epochs import EpochAnimation

from model import Model

import cProfile
import pstats


is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    print("GPU available")
else:
    device = torch.device("cpu")
    print("GPU not available")

device = torch.device("cpu")

In [ ]:
## Load data
data_path = "/home/loek/projects/rnn/DNN/data/"

data = pd.read_csv(data_path + "Rogers McClelland/Table B1.txt", sep=" ", header=0)

# data = data.loc[["Grow", "Move", "Roots", "Fly", "Swim", "Leaves", "Petals"]]
# data = data[["canary", "salmon", "oak", "rose"]]
data.at["Leaves", "rose"] = 0

n = len(data.columns)
inputs = np.array([[1 if j == i else 0 for j in range(n)] for i in range(n)])
outputs = np.array([list(data[index]) for index in data])


# Setup training data
inputs = torch.from_numpy(inputs.astype(np.float32)).to(device)
outputs = torch.from_numpy(outputs.astype(np.float32)).to(device)


dataset = TensorDataset(inputs, outputs)

train_datasets = [dataset]
val_dataset = [dataset]

tracked_datasets = val_dataset + train_datasets

encoding = Direct()

In [ ]:
## Load data
data_path = "/home/loek/projects/rnn/DNN/data/"
properties = pd.read_csv(data_path + "amazing data/Properties.txt", sep=" ", header=0)
classes = pd.read_csv(data_path + "amazing data/Class.txt", sep=" ", header=0)
names = properties.columns.to_numpy()

encoding = OneHot(names.tolist())

n = len(data.columns)
inputs = np.array([list(properties[index]) for index in properties])
inputs = encoding(names)
# inputs = np.array([[1 if j == i else 0 for j in range(n)] for i in range(n)])
outputs = np.array([list(classes[index]) for index in classes])
output_names = dict(zip(classes.index.to_list(), outputs))

# Setup training data
inputs = torch.from_numpy(inputs.astype(np.float32)).to(device)
outputs = torch.from_numpy(outputs.astype(np.float32)).to(device)

dataset = TensorDataset(inputs, outputs)
train_datasets = [dataset]
val_dataset = [dataset]

tracked_datasets = val_dataset + train_datasets

In [ ]:
gain = 1.4

In [ ]:
## Instantiate model
model = Model(
    encoding=encoding,
    input_size=inputs.shape[1],
    output_size=outputs.shape[1],
    hidden_dim=50,
    n_hid_layers=100,
    device=device,
    init_std=gain,
)

In [ ]:
## Setup compiler

# Define hyperparameters
n_epochs = 1000
lr = 0.00005

# Define Loss, Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
compiler = Compiler(model, criterion, optimizer)
compiler.trackers = {
    "loss": ScalarTracker(lambda: compiler.validation(tracked_datasets)),
    "hidden": ActivationTracker(
        model, lambda inputs: model(inputs)[1][int(len(model) / 2)]
    ),
    "hidden_early": ActivationTracker(
        model, lambda inputs: model(inputs)[1][int(len(model) * (1 / 10))]
    ),
    "hidden_late": ActivationTracker(
        model, lambda inputs: model(inputs)[1][int(len(model) * (9 / 10))]
    ),
    "output": ActivationTracker(model, lambda inputs: model(inputs)[0]),
}

In [ ]:
## Training run
compiler.training_run(
    train_datasets, tracked_datasets, n_epochs=n_epochs, batch_size=100
)

In [ ]:
## Get all activations
activations = []

for layer in range(len(model) - 1):
    act = get_activations(val_dataset, lambda inputs: model(inputs)[1][layer], encoding)
    activations.append(act)

index_names = activations[0].index.names
activations = pd.concat(activations, keys=list(range(len(activations))))
activations.index = activations.index.set_names(["Layer"] + index_names)

In [ ]:
animation = SliderAnimation(
    [ActivationsAnimation(activations, transform="PCA", plot_labels=True)],
    parameters=list(set(activations.index.get_level_values("Layer"))),
    parameter_name="Layer",
    fig_size=4,
)

In [ ]:
## Visualize representation dynamics
data_hid_early = compiler.trackers["hidden_early"].get_trace()
data_hid_late = compiler.trackers["hidden_late"].get_trace()
data_output = compiler.trackers["output"].get_trace()
query = "Epoch >= 0"
data_hid_early = data_hid_early.query(query).copy()
data_hid_late = data_hid_late.query(query).copy()
data_output = data_output.query(query).copy()

loss = compiler.trackers["loss"].get_trace()
val_loss = loss.query("Dataset==0")[0].to_numpy()
train_loss = loss.query("Dataset>0").groupby("Epoch").mean()

# weight_change = compiler.trackers["weight change"].get_trace().to_numpy().reshape(-1)

animation = SliderAnimation(
    [
        ActivationsAnimation(
            data_hid_early.query("Dataset != 0"), transform="PCA", plot_labels=True
        ),
        ActivationsAnimation(
            data_hid_late.query("Dataset != 0"), transform="PCA", plot_labels=True
        ),
        ActivationsAnimation(
            data_output.query("Dataset != 0"),
            transform="PCA",
            plot_labels=True,
            fixed_points=output_names,
        ),
        EpochAnimation(
            graphs={
                "Training loss": train_loss,
                "Validation loss": val_loss,
            },
            unitless_graphs={
                # "weight change": weight_change,
            },
            # x_bounds=(0, 800),
            # y_bounds=(0, 1),
        ),
    ],
    parameters=list(set(data_output.index.get_level_values("Epoch"))),
    parameter_name="Epoch",
    fig_size=4,
)

In [ ]:
def to_ndarray(df: pd.DataFrame) -> tuple[np.ndarray, list[np.ndarray]]:
    """
    Convert a multiindexed pandas dataframe to a multidimensional array.

    Returns
    -------
    array : np.ndarray
        An array containting the dataframe entries
        each dimensions of the array corresponds to an index of the dataframe
        and the final dimension corresponding to the column index
    """
    if df.index.nlevels == 1:
        labels = np.array(df.index.get_level_values(level=0))
        return df.to_numpy(), labels

    a = []
    prev_labels = []
    for x in df.groupby(level=0):
        data = x[1].droplevel(level=0)
        array, label = to_ndarray(data)

        prev_labels.append(label)
        a.append(to_ndarray(data))
    prev_labels = np.array(prev_labels)

    labels_this_level = np.array(df.index.get_level_values(level=0))
    labels = [labels_this_level] + prev_labels
    a = np.array(a)
    return a, labels

In [53]:
def f(df):
    nlevels = data_output.index.nlevels
    for level in range(nlevels):
        label = df.copy()
        for index, entry in df.iterrows():
            df.loc[index] = index[level]
    return df

In [60]:
data_output.index.nlevels

3

In [50]:
pd.DataFrame(data_output.index)

,0
0,"(0, 0, pine)"
1,"(0, 0, oak)"
2,"(0, 0, rose)"
3,"(0, 0, daisy)"
4,"(0, 0, robin)"
...,...
523,"(32, 1, daisy)"
524,"(32, 1, robin)"
525,"(32, 1, canary)"
526,"(32, 1, sunfish)"


In [54]:
f(data_output)

0     1     2     3
Epoch Dataset Input                          
0     0       pine      0.0   0.0   0.0   0.0
              oak       0.0   0.0   0.0   0.0
              rose      0.0   0.0   0.0   0.0
              daisy     0.0   0.0   0.0   0.0
              robin     0.0   0.0   0.0   0.0
...                     ...   ...   ...   ...
32    1       daisy    32.0  32.0  32.0  32.0
              robin    32.0  32.0  32.0  32.0
              canary   32.0  32.0  32.0  32.0
              sunfish  32.0  32.0  32.0  32.0
              salmon   32.0  32.0  32.0  32.0

[528 rows x 4 columns]

In [39]:
data_output

0         1         2         3
Epoch Dataset Input                                          
0     0       pine     0.012265 -0.000088 -0.000026 -0.000170
              oak      0.005327 -0.000037 -0.000012 -0.000070
              rose     0.006516 -0.000046 -0.000017 -0.000079
              daisy    0.013585 -0.000096 -0.000033 -0.000168
              robin    0.003689 -0.000025 -0.000008 -0.000048
...                         ...       ...       ...       ...
32    1       daisy    0.156496 -0.000211  0.106585 -0.000823
              robin    0.082774 -0.000076  0.056388 -0.000424
              canary   0.141233 -0.000185  0.096792 -0.000740
              sunfish  0.730763 -0.001069  0.507106 -0.003590
              salmon   0.621237 -0.000902  0.433807 -0.003036

[528 rows x 4 columns]

In [ ]:
to_ndarray(data_output)

In [ ]:
np.array(data_output.index.get_level_values(level=0))

In [ ]:
# name = f"compl{freq}gain{gain}"
name = "Partial Out-of-dataset generalization"
# animation.to_gif("plots/" + name, step_size=int(len(train_loss) / 1000))